#Requirements
Pillow
selenium
requests

In [23]:
import selenium
import time
import io
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import requests
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [108]:
wd = webdriver.Chrome()


In [109]:
wd.get("https://images.google.com/")

In [46]:
search_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/c/c1/Team_Korea_Russia_WorldCup_02_%28cropped%29.png/640px-Team_Korea_Russia_WorldCup_02_%28cropped%29.png"
#search_url = "https://www.google.com/search?q=son+heung+min&sca_esv=37563e55355f6dda&sca_upv=1&source=hp&biw=1280&bih=580&ei=j2PTZtCmBb3Xi-gP-aeBEA&iflsig=AL9hbdgAAAAAZtNxn18OyyYMv8cJizK4LoG0lFO6e_oX&oq=son+he&gs_lp=EgNpbWciBnNvbiBoZSoCCAAyCxAAGIAEGLEDGIMBMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESOQgUO4EWOIZcAZ4AJABAJgBAKABAKoBALgBA8gBAPgBAYoCC2d3cy13aXotaW1nmAIGoAIYqAIAwgIEEAAYA8ICCBAAGIAEGLEDwgIOEAAYgAQYsQMYgwEYigWYAwGSBwE2oAcA&sclient=img&udm=2"
wd.get(search_url)

In [126]:
def scrape_images(wd,delay,max_images):
    def scroll_down(wd):
        wd.execute_script("windows.scrollTo(0,document.body.scrollHeight);")
        time.sleep(1)
    url = "https://www.google.com/search?sca_esv=db73fa7fbeeadfbb&sca_upv=1&rlz=1C1CHBF_deDE1082DE1083&q=kai+havertz&udm=2&fbs=AEQNm0A6bwEop21ehxKWq5cj-cHa02QUie7apaStVTrDAEoT1KC8RoCil8tGAcNCxzRkvcnGwqyGuTD7X18A_UTKd0_CELfTzO-ikZ95qZ2gBUQP1uyQ-pZ_Zw5KqTGBFqE6bltnqKUQNGWnYfdmillxbjYIVPk4_2RXlUx27uc0EROUT1MM3kg&sa=X&ved=2ahUKEwivgZbUn6CIAxV0zgIHHc5FK58QtKgLegQIDRAB&biw=1280&bih=580&dpr=1.5"
    wd.get(url)

    image_urls = set()

    while len(image_urls) <max_images:
        #scroll_down(wd)
        time.sleep(2)
        thumbnails = wd.find_elements(By.CLASS_NAME,"H8Rx8c")
        print("found",len(thumbnails))
        for img in thumbnails[0:100]:
            print(img)
            try:
                img.click()
                time.sleep(2)
            except Exception as e:
                print("Click Failed",e)

            images = wd.find_elements(By.CSS_SELECTOR,"img.sFlh5c.FyHeAf.iPVvYb")
            for image in images:
                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))

        return image_urls

In [127]:
def download_image(download_path,url,file_name):
    try:
        image_content = requests.get(url).content
        print(image_content)
        #Storing the resulting content in computer's memory
        image_file = io.BytesIO(image_content)
        print("writing to memory success")
        #Opening the image with Pillow 
        try:
            image = Image.open(image_file)
            #image.show()  # This opens the image using the default image viewer
            print("Image opened successfully:", image)
        except IOError as e:
            print(f"Error opening image: {e}")
        file_path = download_path + file_name
        print(file_path)

        with open(file_path,"wb") as f: #In writebytes mode
             image.save(f,"JPEG")
        print("success")
    except Exception as e:
         print("Failed Saving", e)

In [ ]:
img_src = scrape_images(wd,1,75)
for i, src in enumerate(img_src):
    if ".png" in src:
        print('its a png image')
        download_image("D:\\Study\\Projects\\ImageClassification\\PremierLeaguePlayersClassifier\\Images\\Havertz\\",src,"image"+str(i)+".png")
    else:
        download_image("D:\\Study\\Projects\\ImageClassification\\PremierLeaguePlayersClassifier\\Images\\Havertz\\",src,"image"+str(i)+".jpg")

In [41]:
wd.quit()